In [5]:
#The purpose of this is to be able to compare the volatility regimes with the existing portfolio framework
#To utilize this, simple run whatever model you want, then upload the vol into the vol_regime.xlsx spreadsheet
#Instead of doing it's own MSM, the model will pull the volatility regime from the excel file and then apply all the logic going forward

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from fredapi import Fred
import sqlite3
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import pyfolio as pf
import empyrical as ep
from decimal import Decimal

def main():
    # Set up your FRED API key
    fred_api_key = 'bf9400b3f6a177d421bda60a77384789'  # Replace with your FRED API key
    fred = Fred(api_key=fred_api_key)
    
    # Define ticker and date range
    ticker = "^GSPC"
    edited_ticker = ticker.replace("^", "")
    start_date = "1950-01-01"
    end_date = (get_previous_trading_day() + timedelta(days=1)).strftime('%Y-%m-%d')
    
    # Fetch market data
    market_data = fetch_market_data(ticker, start_date, end_date)
    
    # Fetch FRED data
    fred_data = fetch_fred_data(start_date, end_date, fred)
    
    # Process data
    data = process_data(market_data, fred_data)
    
    # Read Vol_Regime from Excel file
    data = read_vol_regime_from_excel(data)
    
    # Define market regimes
    data = define_market_regimes(data)
    
    # Add Triangular Moving Averages and Indicators
    data = add_triangular_moving_averages_and_indicators(data)
    
    # Calculate exposures
    data = calculate_exposures(data)
    
    # Calculate returns and portfolio values
    data = calculate_returns_and_portfolio_values(data)
    
    # Calculate performance metrics
    stats_df = calculate_performance_metrics(data)
    
    # Define output paths
    output_csv_path = 'output/output_table.csv'
    output_pdf_path = 'output/output_report.pdf'
    trades_csv_path = 'output/trades_data.csv'
    worst_periods_csv_path = 'output/worst_periods.csv'
    
    # Output results
    output_results(
        data,
        output_csv_path,
        output_pdf_path,
        trades_csv_path,
        worst_periods_csv_path,
        stats_df,
        edited_ticker,
        start_date,
        end_date
    )
    
    # Output to SQLite database
    output_to_database(data)

def get_previous_trading_day():
    today = datetime.now().date()
    previous_day = today - timedelta(days=1)
    
    while previous_day.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
        previous_day -= timedelta(days=1)
    
    return previous_day

def fetch_market_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk")
    
    # Create 'Syn_Open' column
    data['Syn_Open'] = data['Open']
    mask = (data['Open'] == 0) | (data['Open'].isna())
    data.loc[mask, 'Syn_Open'] = data.loc[mask, ['High', 'Low', 'Close']].mean(axis=1)
    
    # Use 'Syn_Open' where 'Open' is NaN or 0
    data['Adjusted_Open'] = data['Syn_Open']
    
    # Calculate daily returns and set the first day's return to 0
    data['Index_Returns'] = data['Adj Close'].pct_change().fillna(0)
    
    return data

def fetch_fred_data(start_date, end_date, fred):
    ffr_daily = fred.get_series('FEDFUNDS', start_date, end_date)
    ffr_10yr = fred.get_series('DGS10', start_date, end_date)
    tb3ms = fred.get_series('TB3MS', start_date, end_date)
    effr_daily = fred.get_series('EFFR', start_date, end_date)
    
    # Convert to DataFrame
    ffr_daily = pd.DataFrame(ffr_daily, columns=['FEDFUNDS'])
    ffr_10yr = pd.DataFrame(ffr_10yr, columns=['DGS10'])
    tb3ms = pd.DataFrame(tb3ms, columns=['TB3MS'])
    effr_daily = pd.DataFrame(effr_daily, columns=['EFFR'])
    
    # Resample to daily frequency and fill missing values
    ffr_daily = ffr_daily.resample('D').ffill()
    ffr_10yr = ffr_10yr.resample('D').ffill()
    tb3ms = tb3ms.resample('D').ffill()
    effr_daily = effr_daily.resample('D').ffill()
    
    fred_data = {
        'FEDFUNDS': ffr_daily,
        'DGS10': ffr_10yr,
        'TB3MS': tb3ms,
        'EFFR': effr_daily
    }
    
    return fred_data

def process_data(data, fred_data):
    # Merge S&P 500 data with Fed Funds Rate data
    data = data.join(fred_data['FEDFUNDS'])
    data = data.join(fred_data['DGS10'])
    data = data.join(fred_data['TB3MS'])
    data = data.join(fred_data['EFFR'])
    
    # Use TB3MS for dates before 1954-07-01, daily rate if available, otherwise use 10-year rate
    data['Effective_Fed_Rate'] = np.where(
        data.index < '1954-07-01',
        data['TB3MS'],
        data['FEDFUNDS']
    )
    data['Effective_Fed_Rate'] = np.where(
        data.index >= '2000-07-03',
        data['EFFR'],
        data['Effective_Fed_Rate']
    )
    data['Effective_Fed_Rate'] = data['Effective_Fed_Rate'].combine_first(data['DGS10'])
    
    # Handle NaN values by using the previous day's value
    data['Effective_Fed_Rate'] = data['Effective_Fed_Rate'].ffill()
    
    # Convert Effective Fed Rate to percentage format
    data['Effective_Fed_Rate'] = data['Effective_Fed_Rate'] / 100
    
    # Define IBKR Fee
    ibkr_fee = 0.0075  # 0.75% as a decimal
    
    # Add IBKR Fee as a new column
    data['IBKR_Rate'] = ibkr_fee
    
    # Calculate Daily Leverage Rate
    data['Daily_Leverage_Rate'] = (data['Effective_Fed_Rate'] + ibkr_fee) / 52
    
    return data

def read_vol_regime_from_excel(data):
    # Read Vol_Regime from vol_regime.xlsx
    vol_regime_df = pd.read_excel('vol_regime.xlsx', index_col=0)
    vol_regime_df.index = pd.to_datetime(vol_regime_df.index)
    
    # Ensure the 'Vol_Regime' column exists
    if 'Vol_Regime' not in vol_regime_df.columns:
        raise ValueError("The 'Vol_Regime' column is not found in vol_regime.xlsx")
    
    # Merge vol_regime_df into data
    data = data.merge(vol_regime_df[['Vol_Regime']], how='left', left_index=True, right_index=True)
    
    return data

def define_market_regimes(data):
    # Calculate 250-day triangular moving average
    data['250_TMA'] = triangular_moving_average(data['Adj Close'], 37)
    
    # Define the four market regimes for 250 TMA
    conditions = [
        (data['Vol_Regime'] == 1) & (data['Adj Close'] < data['250_TMA']),
        (data['Vol_Regime'] == 1) & (data['Adj Close'] >= data['250_TMA']),
        (data['Vol_Regime'] == 0) & (data['Adj Close'] < data['250_TMA']),
        (data['Vol_Regime'] == 0) & (data['Adj Close'] >= data['250_TMA']),
    ]
    choices = [
        'Bearish High Variance',
        'Bullish High Variance',
        'Bearish Low Variance',
        'Bullish Low Variance'
    ]
    
    # Specify a default value that matches the data type of choices
    data['Market_Regime'] = np.select(conditions, choices, default='Unknown')
    
    # Define adjusted market regimes with offset (shifted by 1 day)
    data['Adjusted_Market_Regime'] = data['Market_Regime'].shift(1)
    
    return data

def calculate_exposures(data):
    # Define initial exposure based on Adjusted_Market_Regime
    exposure_mapping = {
        'Bullish Low Variance': 2.0,
        'Bearish Low Variance': 1.0,
        'Bullish High Variance': 1.0,
        'Bearish High Variance': 0.0
    }
    data['Portfolio_Exposure'] = data['Adjusted_Market_Regime'].map(exposure_mapping).fillna(1.0)  # Default exposure is 1.0 if regime is NaN
    
    # Adjust exposure based on 30-Day and 60-Day Indicators
    for index, row in data.iterrows():
        if row['Portfolio_Exposure'] == 2.0:
            if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
            elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.5
            elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
                data.at[index, 'Portfolio_Exposure'] = 1.5
                    
    # Adjust exposure based on 30-Day and 60-Day Indicators for exposure = 1.0 and Bearish Low Variance regime
    for index, row in data.iterrows():
        if row['Portfolio_Exposure'] == 1.0 and row['Adjusted_Market_Regime'] == 'Bearish Low Variance':
            if row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 0.0
            elif row['30_Day_Indicator'] == 'Bullish' and row['60_Day_Indicator'] == 'Bearish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
            elif row['30_Day_Indicator'] == 'Bearish' and row['60_Day_Indicator'] == 'Bullish':
                data.at[index, 'Portfolio_Exposure'] = 1.0
                    
    return data

def calculate_returns_and_portfolio_values(data):
    initial_value = 100000
    
    # Initialize 'Beginning_Portfolio_Value' if it does not exist
    if 'Beginning_Portfolio_Value' not in data.columns:
        data['Beginning_Portfolio_Value'] = initial_value
    
    # Calculate strategy returns and adjust for leverage cost and transaction costs
    data['Leveraged_Portion'] = data['Portfolio_Exposure'] - 1
    data['Leveraged_Portion'] = data['Leveraged_Portion'].apply(lambda x: max(x, 0))  # Only positive leverage
    
    # Adjust leverage cost calculation based on the current portfolio value
    data['Leverage_Cost_Amount'] = data['Beginning_Portfolio_Value'] * data['Leveraged_Portion'] * data['Daily_Leverage_Rate']
    
    # Transaction and Slippage costs calculation
    transaction_cost_per_trade = 0.002  # Example: 0.1% per trade + 0.1% per trade on slippage
    data['Transaction_Slippage_Costs'] = transaction_cost_per_trade * np.abs(data['Portfolio_Exposure'].diff().fillna(0))
    
    # Calculate transaction cost in dollar amounts
    data['Transaction_Cost_Dollars'] = data['Transaction_Slippage_Costs'] * data['Beginning_Portfolio_Value']
    
    shorting_cost = 0.003  # Example: 0.3% for shorting
    data['Shorting_Costs'] = shorting_cost * (data['Portfolio_Exposure'] < 0).astype(int)
    
    # Update the strategy return calculation to use leverage cost directly
    data['Strategy_Return'] = (
        data['Index_Returns'] * data['Portfolio_Exposure']
        - data['Leverage_Cost_Amount'] / data['Beginning_Portfolio_Value']  # Use current portfolio value instead of initial
        - data['Transaction_Slippage_Costs']
        - data['Shorting_Costs']
    )
    
    # Set the strategy return for the first date to 0
    data.at[data.index[0], 'Strategy_Return'] = 0
    
    # Calculate cumulative returns starting with $100,000
    data['Portfolio_Value'] = initial_value * (1 + data['Strategy_Return']).cumprod()
    data['Market_Value'] = initial_value * (1 + data['Index_Returns']).cumprod()
    
    # Calculate the beginning portfolio value for each day
    data['Beginning_Portfolio_Value'] = data['Portfolio_Value'].shift(1).fillna(initial_value)
    
    # Recalculate transaction cost in dollar amounts after 'Beginning_Portfolio_Value' is updated
    data['Transaction_Cost_Dollars'] = data['Transaction_Slippage_Costs'] * data['Beginning_Portfolio_Value']
    
    # Calculate the daily return based on the beginning portfolio value
    data['Daily_Return'] = data['Beginning_Portfolio_Value'] * data['Strategy_Return']
    
    # Identify buy/sell signals based on changes in exposure
    data['Trade_Signal'] = ''
    data['Trade_Signal'] = np.where(data['Portfolio_Exposure'].diff() > 0, 'Buy', data['Trade_Signal'])
    data['Trade_Signal'] = np.where(data['Portfolio_Exposure'].diff() < 0, 'Sell', data['Trade_Signal'])
    data.at[data.index[0], 'Trade_Signal'] = 'Buy'
    
    # Adjust trade signals for next day's open price
    data['Next_Open'] = data['Open'].shift(0)
    data['Trade_Signal_Next_Open'] = data['Trade_Signal'].shift(0)
    
    # Add new columns to find the Beginning Portfolio Value and Date of the last "Buy" signal for each "Sell"
    data['Last_Buy_Value'] = None
    data['Last_Buy_Date'] = None
    last_buy_value = None
    last_buy_date = None

    for i in range(len(data)):
        if data['Trade_Signal_Next_Open'].iloc[i] == 'Buy':
            last_buy_value = data['Next_Open'].iloc[i]
            last_buy_date = data.index[i].date()  # Keep only the date element
        elif data['Trade_Signal_Next_Open'].iloc[i] == 'Sell' and last_buy_value is not None:
            data.at[data.index[i], 'Last_Buy_Value'] = last_buy_value
            data.at[data.index[i], 'Last_Buy_Date'] = last_buy_date  # Keep only the date element

    # Convert 'Last_Buy_Date' to string format to ensure only date is stored, not time.
    data['Last_Buy_Date'] = pd.to_datetime(data['Last_Buy_Date']).dt.date
    
    # Add a new column for Profit/Loss
    def calculate_profit_loss(row):
        if (
            row['Trade_Signal_Next_Open'] == 'Sell'
            and pd.notnull(row['Last_Buy_Value'])
            and pd.notnull(row['Next_Open'])
        ):
            return row['Next_Open'] - row['Last_Buy_Value']
        return 0
    
    data['Profit/Loss'] = data.apply(calculate_profit_loss, axis=1)
    
    def calculate_tax(row):
        if row['Trade_Signal_Next_Open'] == 'Sell' and pd.notnull(row['Last_Buy_Date']):
            # Convert Last_Buy_Date to Timestamp to match row.name type
            last_buy_date = pd.Timestamp(row['Last_Buy_Date'])
            days_held = (row.name - last_buy_date).days
            profit_loss = row['Profit/Loss']
            if days_held > 365:
                return 0.00 * profit_loss
            else:
                return 0.00 * profit_loss
        return 0
    
    data['Tax'] = data.apply(calculate_tax, axis=1)
    
    # Calculate Tax Amount in dollars based on the Profit/Loss when selling
    data['Tax_Amount'] = data['Tax']
    
    # Adjust the ending portfolio value for tax when selling
    data['Ending_Portfolio_Value'] = np.where(
        data['Trade_Signal_Next_Open'] == 'Sell',
        data['Beginning_Portfolio_Value'] + data['Daily_Return'] - data['Tax_Amount'],
        data['Beginning_Portfolio_Value'] + data['Daily_Return']
    )
    
    # Handle NaN in 'Ending_Portfolio_Value' by filling it with the previous value or the initial value
    data['Ending_Portfolio_Value'] = data['Ending_Portfolio_Value'].ffill().fillna(initial_value)
    
    # Calculate the beginning portfolio value for the next day including tax
    data['Beginning_Portfolio_Value'] = data['Ending_Portfolio_Value'].shift(1).fillna(initial_value)
    
    # Calculate drawdowns for the market (index) and the strategy
    data['Index_Drawdown'] = data['Market_Value'] / data['Market_Value'].cummax() - 1
    data['Strategy_Drawdown'] = data['Ending_Portfolio_Value'] / data['Ending_Portfolio_Value'].cummax() - 1
    
    return data

def add_triangular_moving_averages_and_indicators(data):
    # Calculate 30-day and 60-day Triangular Moving Averages and shift by 1 day
    data['30_TMA'] = triangular_moving_average(data['Adj Close'], 4).shift(1)
    data['60_TMA'] = triangular_moving_average(data['Adj Close'], 8).shift(1)
    
    # Define 30-Day and 60-Day Indicators
    data['30_Day_Indicator'] = np.where(data['Adj Close'] > data['30_TMA'], 'Bullish', 'Bearish')
    data['60_Day_Indicator'] = np.where(data['Adj Close'] > data['60_TMA'], 'Bullish', 'Bearish')
    
    return data

def triangular_moving_average(series, n):
    # Calculate the triangular moving average with a two-step rolling mean
    smoothed_series = series.rolling(window=(n // 2), min_periods=1).mean()
    smoothed_series = smoothed_series.rolling(window=(n // 2), min_periods=1).mean()
    return smoothed_series

def calculate_custom_metrics(returns):
    """
    Calculate custom performance metrics that are not directly provided by Pyfolio.
    """
    # Annual return
    annual_return = ep.annual_return(returns)
    
    # Cumulative returns
    cumulative_returns = ep.cum_returns_final(returns)
    
    # Annual volatility
    annual_volatility = ep.annual_volatility(returns)
    
    # Sharpe ratio
    sharpe_ratio = ep.sharpe_ratio(returns)
    
    # Calmar ratio
    calmar_ratio = ep.calmar_ratio(returns)
    
    # Stability
    stability = ep.stability_of_timeseries(returns)
    
    # Max drawdown
    max_drawdown = ep.max_drawdown(returns)
    
    # Omega ratio (using a threshold of 0)
    omega_ratio = ep.omega_ratio(returns, required_return=0)
    
    # Sortino ratio (using a threshold of 0)
    sortino_ratio = ep.sortino_ratio(returns, required_return=0)
    
    # Tail ratio
    tail_ratio = ep.tail_ratio(returns)
    
    # Value at risk (VaR)
    var = ep.value_at_risk(returns)
    
    # Collecting results into a dictionary
    metrics = {
        'Annual Return': f"{annual_return:.2%}",
        'Cumulative Returns': f"{cumulative_returns:.2%}",
        'Annual Volatility': f"{annual_volatility:.2%}",
        'Sharpe Ratio': f"{sharpe_ratio:.2f}",
        'Calmar Ratio': f"{calmar_ratio:.2f}",
        'Stability': f"{stability:.2f}",
        'Max Drawdown': f"{max_drawdown:.2%}",
        'Omega Ratio': f"{omega_ratio:.2f}",
        'Sortino Ratio': f"{sortino_ratio:.2f}",
        'Tail Ratio': f"{tail_ratio:.2f}",
        'Value at Risk (VaR)': f"{var:.2%}"
    }
    
    return metrics

def run_pyfolio_analysis(data):
    """
    Runs a pyfolio tear sheet analysis on the strategy returns and includes custom performance metrics.
    """
    strategy_returns = data['Strategy_Return']
    
    # Convert index to UTC if not already done
    if strategy_returns.index.tz is None:
        strategy_returns.index = strategy_returns.index.tz_localize('UTC')
    
    # Calculate custom metrics
    custom_metrics = calculate_custom_metrics(strategy_returns)
    
    # Create a new PDF file to save plots
    pdf_path = 'pyfolio_output_report.pdf'
    with PdfPages(pdf_path) as pdf:
        
        # Create and save each plot to the PDF as a separate figure
        fig, ax = plt.subplots()
        pf.plot_rolling_returns(strategy_returns, ax=ax)
        pdf.savefig(fig)
        plt.close(fig)

        fig, ax = plt.subplots()
        pf.plot_drawdown_underwater(strategy_returns, ax=ax)
        pdf.savefig(fig)
        plt.close(fig)

        fig, ax = plt.subplots()
        pf.plot_rolling_volatility(strategy_returns, ax=ax)
        pdf.savefig(fig)
        plt.close(fig)

        fig, ax = plt.subplots()
        pf.plot_rolling_sharpe(strategy_returns, ax=ax)
        pdf.savefig(fig)
        plt.close(fig)

        fig, ax = plt.subplots()
        pf.plot_monthly_returns_heatmap(strategy_returns, ax=ax)
        pdf.savefig(fig)
        plt.close(fig)

        fig, ax = plt.subplots()
        pf.plot_annual_returns(strategy_returns, ax=ax)
        pdf.savefig(fig)
        plt.close(fig)

        fig, ax = plt.subplots()
        pf.plot_monthly_returns_dist(strategy_returns, ax=ax)
        pdf.savefig(fig)
        plt.close(fig)
        
        # Add more plots as needed...
    
    print(f"PDF report saved to {pdf_path}")
    
    # Display custom metrics
    print("\nPerformance Metrics:")
    for metric, value in custom_metrics.items():
        print(f"{metric}: {value}")
    
    # Create a DataFrame to display in a similar format as in your image
    metrics_df = pd.DataFrame(custom_metrics.items(), columns=['Metric', 'Value'])
    
    print("\nCustom Metrics Table:")
    print(metrics_df.to_string(index=False))
    
    # Save the DataFrame to a CSV file if needed
    metrics_df.to_csv('performance_metrics.csv', index=False)

def calculate_cagr(portfolio_value):
    # Ensure the index is in datetime format
    portfolio_value.index = pd.to_datetime(portfolio_value.index)

    # Calculate the actual number of years between the first and last date using Decimal
    start_date = portfolio_value.index.min()
    end_date = portfolio_value.index.max()
    total_years = Decimal((end_date - start_date).days) / Decimal(365.25)

    # Calculate CAGR using the actual time period with Decimal
    start_value = Decimal(portfolio_value.iloc[0])
    end_value = Decimal(portfolio_value.iloc[-1])
    cagr_value = (end_value / start_value) ** (Decimal(1) / total_years) - Decimal(1)

    return float(cagr_value)

def calculate_sharpe_ratio(returns, risk_free_rate=0.03, periods_per_year=52):
    excess_returns = returns - risk_free_rate / periods_per_year
    return np.mean(excess_returns) / np.std(excess_returns) * np.sqrt(periods_per_year)

def calculate_sortino_ratio(returns, risk_free_rate=0, periods_per_year=52):
    downside_returns = returns[returns < 0]
    excess_returns = returns - risk_free_rate / periods_per_year
    return np.mean(excess_returns) / np.std(downside_returns) * np.sqrt(periods_per_year)

def calculate_information_ratio(strategy_returns, benchmark_returns, periods_per_year=52):
    # Ensure both series have the same time zone awareness (UTC for both)
    if strategy_returns.index.tz is None:
        strategy_returns.index = strategy_returns.index.tz_localize('UTC')
    if benchmark_returns.index.tz is None:
        benchmark_returns.index = benchmark_returns.index.tz_localize('UTC')

    # Align the indices of both returns series to avoid mismatches
    strategy_returns, benchmark_returns = strategy_returns.align(benchmark_returns, join='inner')

    # Calculate the information ratio
    excess_returns = strategy_returns - benchmark_returns
    return np.mean(excess_returns) / np.std(excess_returns) * np.sqrt(periods_per_year)

def calculate_max_drawdown(portfolio_value):
    rolling_max = portfolio_value.cummax()
    drawdown = portfolio_value / rolling_max - 1
    max_drawdown = drawdown.min()
    drawdown_trough_date = drawdown.idxmin()
    drawdown_peak_date = rolling_max[:drawdown_trough_date].idxmax()
    drawdown_breakeven_date = (portfolio_value[drawdown_trough_date:] >= rolling_max[drawdown_peak_date]).idxmax()
    return max_drawdown, drawdown_peak_date, drawdown_trough_date, drawdown_breakeven_date

def calculate_performance_metrics(data):
    # Calculate strategy and market statistics
    strategy_cagr = calculate_cagr(data['Portfolio_Value'])
    market_cagr = calculate_cagr(data['Market_Value'])
    strategy_max_drawdown, strategy_peak_date, strategy_trough_date, strategy_breakeven_date = calculate_max_drawdown(data['Portfolio_Value'])
    market_max_drawdown, _, _, _ = calculate_max_drawdown(data['Market_Value'])
    strategy_sharpe_ratio = round(calculate_sharpe_ratio(data['Strategy_Return']), 2)
    market_sharpe_ratio = round(calculate_sharpe_ratio(data['Index_Returns']), 2)
    strategy_sortino_ratio = round(calculate_sortino_ratio(data['Strategy_Return']), 2)
    market_sortino_ratio = round(calculate_sortino_ratio(data['Index_Returns']), 2)
    information_ratio = round(calculate_information_ratio(data['Strategy_Return'], data['Index_Returns']), 2)
    
    # Calculate the percentage of time the Portfolio Exposure is >= 1
    exposure_percentage = (data['Portfolio_Exposure'] >= 1).mean() * 100
    exposure_percentage_str = f'{exposure_percentage:.2f}%'
    
    # Format CAGR and Max Drawdown as percentage with 2 decimal places
    strategy_cagr_percent = f'{strategy_cagr * 100:.10f}%'
    market_cagr_percent = f'{market_cagr * 100:.10f}%'
    strategy_max_drawdown_percent = f'{strategy_max_drawdown * 100:.2f}%'
    market_max_drawdown_percent = f'{market_max_drawdown * 100:.2f}%'
    
    # Create the Performance Statistics DataFrame
    stats_df = pd.DataFrame({
        'Statistic': [
            'CAGR',
            'Max Drawdown',
            'Sharpe Ratio',
            'Sortino Ratio',
            'Information Ratio',
            'Exposure >= 1'
        ],
        'Strategy': [
            strategy_cagr_percent,
            strategy_max_drawdown_percent,
            strategy_sharpe_ratio,
            strategy_sortino_ratio,
            information_ratio,
            exposure_percentage_str
        ],
        'Market': [
            market_cagr_percent,
            market_max_drawdown_percent,
            market_sharpe_ratio,
            market_sortino_ratio,
            np.nan,
            np.nan
        ]
    })
    
    return stats_df

def output_results(data, output_csv_path, output_pdf_path, trades_csv_path, worst_periods_csv_path, stats_df, edited_ticker, start_date, end_date):
    # Format the output table after all calculations
    formatted_data = data.copy()
    
    # Display the performance statistics
    print("\nPerformance Statistics:")
    print(stats_df.to_string(index=False))
    
    # Define the desired column order
    column_order = [
        'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Syn_Open', 'Adjusted_Open',
        'FEDFUNDS', 'DGS10', 'TB3MS', 'EFFR', 'Effective_Fed_Rate', 'IBKR_Rate', 'Daily_Leverage_Rate',
        'Vol_Regime', '250_TMA', 'Market_Regime', 'Adjusted_Market_Regime', '30_TMA', '60_TMA',
        '30_Day_Indicator', '60_Day_Indicator', 'Leveraged_Portion', 'Beginning_Portfolio_Value',
        'Index_Returns', 'Portfolio_Exposure', 'Leverage_Cost_Amount', 'Transaction_Slippage_Costs', 'Transaction_Cost_Dollars',
        'Shorting_Costs', 'Strategy_Return', 'Daily_Return', 'Trade_Signal', 'Next_Open', 'Trade_Signal_Next_Open',
        'Last_Buy_Value', 'Last_Buy_Date', 'Profit/Loss', 'Tax', 'Tax_Amount', 'Ending_Portfolio_Value',
        'Index_Drawdown', 'Strategy_Drawdown', 'Portfolio_Value', 'Market_Value'
    ]
    
    # Reorder the columns in the data
    data = data[column_order]
    
    # Save the combined output table to a CSV file
    data.to_csv(output_csv_path, index=True)
    
    # Display the output table in the console
    print("Output table:\n", data)
    
    # Create the trades output CSV with open and close values and trade signals
    trades_data = data[['Open', 'Close', 'Trade_Signal', 'Portfolio_Exposure']].copy()
    trades_data['Next_Day_Open'] = trades_data['Open'].shift(-1)
    trades_data = trades_data.dropna(subset=['Trade_Signal'])
    
    # Save the trades data to a CSV file
    trades_data.to_csv(trades_csv_path, index=True)
    
    # Define helper functions
    def find_nearest_date(target_date, date_index):
        """
        Find the nearest date in date_index to the target_date
        """
        nearest_date = min(date_index, key=lambda x: abs(x - target_date))
        return nearest_date

    def plot_3yr_comparisons(data, worst_periods, pdf):
        for i, row in worst_periods.iterrows():
            start_date = row['Start_Date']
            end_date = row['End_Date']
            period_data = data.loc[start_date:end_date]
    
            # Normalize starting values to 100
            normalized_strategy = (period_data['Portfolio_Value'] / period_data['Portfolio_Value'].iloc[0]) * 100
            normalized_market = (period_data['Market_Value'] / period_data['Market_Value'].iloc[0]) * 100
    
            fig = plt.figure(figsize=(12, 8))
            plt.plot(period_data.index, normalized_strategy, label='Strategy (normalized)')
            plt.plot(period_data.index, normalized_market, label='Market Index (normalized)')
            plt.title(f'Price Return Comparison from {start_date.date()} to {end_date.date()}')
            plt.xlabel('Date')
            plt.ylabel('Normalized Price Return')
            plt.legend()
            plt.grid(True)
            plt.xticks(rotation=45, ha='right')
            pdf.savefig(fig)
            plt.close()
    
    # Save figures to a PDF report
    with PdfPages(output_pdf_path) as pdf:
        # Add a cover page
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.axis('off')
        cover_title = edited_ticker
        end_date_minus_one = pd.to_datetime(end_date) - timedelta(days=1)
        cover_date_range = f"{pd.to_datetime(start_date).strftime('%m/%d/%Y')} - {end_date_minus_one.strftime('%m/%d/%Y')}"
        ax.text(0.5, 0.7, cover_title, fontsize=26, ha='center', va='center', fontname='Times New Roman')
        ax.text(0.5, 0.5, cover_date_range, fontsize=16, ha='center', va='center', fontname='Times New Roman')
        pdf.savefig(fig)
        plt.close()
    
        # Add performance statistics table
        fig = plt.figure(figsize=(12, 4))
        gs = gridspec.GridSpec(1, 1)
    
        # Add performance statistics
        ax0 = plt.subplot(gs[0])
        ax0.axis('tight')
        ax0.axis('off')
        table = ax0.table(cellText=stats_df.round(6).values,
                          colLabels=stats_df.columns,
                          cellLoc='center',
                          loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.2)
        ax0.set_title('Performance Statistics', pad=20)
    
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()
        
        # Plot the S&P 500 price with regimes highlighted
        fig, ax = plt.subplots(figsize=(12, 6))
    
        # Plot each regime segment separately without overlapping
        colors = {
            'Bearish High Variance': 'red',
            'Bullish High Variance': 'green',
            'Bearish Low Variance': 'orange',
            'Bullish Low Variance': 'blue'
        }
    
        for regime, regime_data in data.groupby((data['Market_Regime'] != data['Market_Regime'].shift()).cumsum()):
            regime_label = regime_data['Market_Regime'].iloc[0]
            ax.plot(regime_data.index, regime_data['Adj Close'], color=colors.get(regime_label, 'black'), label=regime_label, linewidth=1.35)
    
        handles, labels = ax.get_legend_handles_labels()
        unique_labels = dict(zip(labels, handles))
        ax.legend(unique_labels.values(), unique_labels.keys())
    
        ax.set_title('Market Regimes Visualized')
        ax.set_xlabel('Date')
        ax.set_ylabel('Adjusted Close Price')
        ax.grid(True)
        ax.set_yscale('log')  # Set y-axis to log scale
        years = pd.date_range(start=start_date, end=end_date, freq='5YS' if (pd.to_datetime(end_date).year - pd.to_datetime(start_date).year) > 25 else 'YS').year
        ax.set_xticks(pd.to_datetime(years, format='%Y'))
        ax.set_xticklabels([f"'{str(year)[-2:]}" for year in years])
    
        for label in ax.get_xticklabels():
            label.set_rotation(45)
            label.set_ha('right')
    
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()
    
        # Create subplots for final market regimes visualization
        fig, axes = plt.subplots(3, 1, figsize=(18, 18), gridspec_kw={'height_ratios': [2, 1, 1]})
    
        # Plot the S&P 500 price with distinct market regimes highlighted
        ax1 = axes[0]
        for regime, regime_data in data.groupby((data['Market_Regime'] != data['Market_Regime'].shift()).cumsum()):
            regime_label = regime_data['Market_Regime'].iloc[0]
            ax1.plot(regime_data.index, regime_data['Adj Close'], color=colors.get(regime_label, 'black'), label=regime_label, linewidth=1.35)
    
        handles, labels = ax1.get_legend_handles_labels()
        unique_labels = dict(zip(labels, handles))
        ax1.legend(unique_labels.values(), unique_labels.keys())
    
        ax1.set_title('Final Market Regimes Visualized')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Adjusted Close Price')
        ax1.grid(True)
        ax1.set_yscale('log')  # Set y-axis to log scale
        years = pd.date_range(start=start_date, end=end_date, freq='5YS' if (pd.to_datetime(end_date).year - pd.to_datetime(start_date).year) > 25 else 'YS').year
        ax1.set_xticks(pd.to_datetime(years, format='%Y'))
        ax1.set_xticklabels([f"'{str(year)[-2:]}" for year in years])
    
        for label in ax1.get_xticklabels():
            label.set_rotation(45)
            label.set_ha('right')
    
        # Calculate regime lengths
        data.loc[:, 'Regime_Change'] = data['Market_Regime'] != data['Market_Regime'].shift()
        data.loc[:, 'Regime_ID'] = data['Regime_Change'].cumsum()
        regime_lengths = data.groupby('Regime_ID').size()
    
        # Calculate descriptive statistics for regime lengths
        regime_length_stats = regime_lengths.describe()
        print("Regime Length Statistics:\n", regime_length_stats)
    
        # Calculate returns statistics for each regime
        returns_stats = data.groupby('Market_Regime')['Index_Returns'].agg(['mean', 'std', 'count'])
        returns_stats['mean_regime_length'] = regime_lengths.groupby(data['Market_Regime']).mean()
    
        # Rename columns for clarity
        returns_stats.columns = ['Mean Daily Return', 'Daily Standard Deviation', 'Number of Days', 'Mean Regime Length']
    
        # Convert returns to percentage
        returns_stats['Mean Daily Return'] *= 100
        returns_stats['Daily Standard Deviation'] *= 100
    
        print("\nReturns Statistics by Market Regime:\n", returns_stats)
    
        # Plot histogram of regime lengths
        ax2 = axes[1]
        ax2.hist(regime_lengths, bins=30, edgecolor='black')
        ax2.set_title('Histogram of Regime Lengths')
        ax2.set_xlabel('Regime Length')
        ax2.set_ylabel('Frequency')
        ax2.title.set_position([.5, 1.05])
    
        # Print returns statistics as a table
        ax3 = axes[2]
        ax3.axis('tight')
        ax3.axis('off')
        table = ax3.table(cellText=returns_stats.round(3).values,
                          colLabels=returns_stats.columns,
                          rowLabels=returns_stats.index,
                          cellLoc='center',
                          loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(12)
        table.scale(1.2, 1.2)
        ax3.set_title('Returns Statistics by Market Regime', pad=20)
    
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        pdf.savefig(fig)
        plt.close()
    
        # Plot the portfolio value on the first subplot
        fig, axes = plt.subplots(2, 1, figsize=(12, 12))
    
        ax1 = axes[0]
        ax1.plot(data.index, data['Portfolio_Value'], label='Strategy Portfolio', color='blue')
        ax1.set_title('Portfolio Value Over Time')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Portfolio Value')
        ax1.grid(True)
        ax1.legend()
    
        # Plot the market value on the second subplot
        ax2 = axes[1]
        ax2.plot(data.index, data['Market_Value'], label='Market (S&P 500)', color='black')
        ax2.set_title('Market Value Over Time')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Market Value')
        ax2.grid(True)
        ax2.legend()
    
        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()
    
        # Plotting the 3-year comparisons
        def rolling_3yr_cagr(series):
            window = 252 * 3  # 252 trading days per year
            return series.pct_change(window, fill_method=None).apply(lambda x: (1 + x) ** (1 / 3) - 1 if pd.notnull(x) else np.nan)
    
        data.loc[:, 'Market_3yr_CAGR'] = rolling_3yr_cagr(data['Market_Value'])
        data.loc[:, 'Strategy_3yr_CAGR'] = rolling_3yr_cagr(data['Portfolio_Value'])
    
        def worst_3yr_per_decade(data):
            worst_periods = []
            for decade in range(1960, 2030, 10):
                start_decade = f'{decade}-01-01'
                end_decade = f'{decade + 9}-12-31'
                decade_data = data.loc[start_decade:end_decade]
                if not decade_data.empty:
                    worst_period = decade_data.nsmallest(1, 'Market_3yr_CAGR')[['Market_3yr_CAGR', 'Strategy_3yr_CAGR']]
                    if not worst_period.empty:
                        worst_period['End_Date'] = worst_period.index
                        worst_period['Start_Date'] = worst_period['End_Date'] - pd.DateOffset(years=3)
                        worst_periods.append(worst_period)
            return pd.concat(worst_periods)
    
        worst_periods_decade = worst_3yr_per_decade(data)
        worst_periods_decade['Start_Date'] = worst_periods_decade['Start_Date'].apply(lambda x: find_nearest_date(x, data.index))
        worst_periods_decade['End_Date'] = worst_periods_decade['End_Date'].apply(lambda x: find_nearest_date(x, data.index))
    
        worst_periods_decade['Start_Strategy_Value'] = [data.loc[start, 'Portfolio_Value'] for start in worst_periods_decade['Start_Date']]
        worst_periods_decade['End_Strategy_Value'] = [data.loc[end, 'Portfolio_Value'] for end in worst_periods_decade['End_Date']]
        worst_periods_decade['Start_Market_Value'] = [data.loc[start, 'Market_Value'] for start in worst_periods_decade['Start_Date']]
        worst_periods_decade['End_Market_Value'] = [data.loc[end, 'Market_Value'] for end in worst_periods_decade['End_Date']]
    
        worst_periods_decade = worst_periods_decade[['Start_Date', 'End_Date', 'Market_3yr_CAGR', 'Strategy_3yr_CAGR',
                                                     'Start_Strategy_Value', 'End_Strategy_Value',
                                                     'Start_Market_Value', 'End_Market_Value']]
    
        # Save the worst periods to a CSV file
        worst_periods_decade.to_csv(worst_periods_csv_path, index=False)
    
        # Display the worst periods in the console
        print("\nWorst 3-Year Periods for Each Decade (Market Downturns):")
        print(worst_periods_decade)
    
        # Plotting the 3-year comparisons
        plot_3yr_comparisons(data, worst_periods_decade, pdf)
    
        # Combine current info with the last 30 days exposure and leverage, and market regime
        exposure_leverage_30_days = data[['Portfolio_Exposure', 'Leveraged_Portion', 'Market_Regime']].tail(30).reset_index()
        exposure_leverage_30_days.columns = ['Date', 'Portfolio Exposure', 'Portfolio Leverage', 'Vol_Regime']
        exposure_leverage_30_days['Date'] = exposure_leverage_30_days['Date'].dt.strftime('%m-%d-%Y')
    
        current_info = {
            'Date': ['Current'],
            'Portfolio Exposure': [data['Portfolio_Exposure'].iloc[-1]],
            'Portfolio Leverage': [data['Leveraged_Portion'].iloc[-1]],
            'Vol_Regime': [data['Market_Regime'].iloc[-1]]
        }
        current_info_df = pd.DataFrame(current_info)
    
        combined_df = pd.concat([current_info_df, exposure_leverage_30_days], ignore_index=True)
    
        fig, ax = plt.subplots(figsize=(12, 8))
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=combined_df.values,
                         colLabels=combined_df.columns,
                         cellLoc='center',
                         loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8.5)
        table.scale(.90, .90)
        ax.set_title('Current Regime, Portfolio Exposure, and Leverage and Market Regime Over the Past 30 Business Days', pad=20)
        fig.subplots_adjust(left=0.1, right=0.9)
    
        pdf.savefig(fig)
        plt.close()
    
    print("\nPerformance Statistics:")
    print(stats_df.to_string(index=False))
    
    # Display the combined table in the console
    print("\nCombined Table:")
    print(combined_df)
    
def output_to_database(data):
    # Define the desired column order
    column_order = [
        'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Syn_Open', 'Adjusted_Open',
        'FEDFUNDS', 'DGS10', 'TB3MS', 'EFFR', 'Effective_Fed_Rate', 'IBKR_Rate', 'Daily_Leverage_Rate',
        'Vol_Regime', '250_TMA', 'Market_Regime', 'Adjusted_Market_Regime', '30_TMA', '60_TMA',
        '30_Day_Indicator', '60_Day_Indicator', 'Leveraged_Portion', 'Beginning_Portfolio_Value',
        'Index_Returns', 'Portfolio_Exposure', 'Leverage_Cost_Amount', 'Transaction_Slippage_Costs', 'Transaction_Cost_Dollars',
        'Shorting_Costs', 'Strategy_Return', 'Daily_Return', 'Trade_Signal', 'Next_Open', 'Trade_Signal_Next_Open',
        'Last_Buy_Value', 'Last_Buy_Date', 'Profit/Loss', 'Tax', 'Tax_Amount', 'Ending_Portfolio_Value',
        'Index_Drawdown', 'Strategy_Drawdown', 'Portfolio_Value', 'Market_Value'
    ]
    
    # Reorder the columns in the data
    data = data[column_order]
    
    # Reset index to ensure it's a column and not an index
    data_reset = data.reset_index()

    # Convert all columns of datetime64 dtype to string format
    for col in data_reset.select_dtypes(include=['datetime64[ns]', 'datetime64']).columns:
        data_reset[col] = data_reset[col].dt.strftime('%Y-%m-%d %H:%M:%S')

    # Convert all 'object' dtype columns containing datetime-like objects to strings
    for col in data_reset.select_dtypes(include=['object']).columns:
        if isinstance(data_reset[col].iloc[0], (pd.Timestamp, datetime)):
            data_reset[col] = data_reset[col].astype(str)

    # Iterate through all columns and convert datetime-like objects to strings
    for col in data_reset.columns:
        data_reset[col] = data_reset[col].apply(lambda x: str(x) if isinstance(x, (pd.Timestamp, datetime)) else x)

    # Output to SQLite database
    with sqlite3.connect('output/financial_model.db') as conn:
        try:
            data_reset.to_sql('financial_data', conn, if_exists='replace', index=False)
        except Exception as e:
            print(f"An error occurred while writing to the database: {e}")
            print("Data types of DataFrame columns:")
            print(data_reset.dtypes)
            print("First few rows of DataFrame:")
            print(data_reset.head())
            
if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed


C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_47776\1952905365.py:537: RuntimeWarning: invalid value encountered in double_scalars
  return np.mean(excess_returns) / np.std(excess_returns) * np.sqrt(periods_per_year)



Performance Statistics:
        Statistic       Strategy         Market
             CAGR 11.6246247249% 11.6246247249%
     Max Drawdown        -29.18%        -29.18%
     Sharpe Ratio           0.58           0.58
    Sortino Ratio           0.85           0.85
Information Ratio            NaN            NaN
    Exposure >= 1        100.00%            NaN
Output table:
                    Open         High          Low        Close    Adj Close  \
Date                                                                          
2015-01-01  2058.899902  2072.360107  1992.439941  2025.900024  2025.900024   
2015-01-08  2030.609985  2064.429932  1988.439941  2011.270020  2011.270020   
2015-01-15  2013.750000  2038.290039  1988.119995  2032.119995  2032.119995   
2015-01-22  2034.300049  2064.620117  2001.489990  2002.160034  2002.160034   
2015-01-29  2002.449951  2054.739990  1980.900024  2041.510010  2041.510010   
...                 ...          ...          ...          ...         

C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_47776\1952905365.py:764: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Regime_Change'] = data['Market_Regime'] != data['Market_Regime'].shift()
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_47776\1952905365.py:765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Regime_ID'] = data['Regime_Change'].cumsum()


Regime Length Statistics:
 count      1.0
mean     515.0
std        NaN
min      515.0
25%      515.0
50%      515.0
75%      515.0
max      515.0
dtype: float64

Returns Statistics by Market Regime:
                Mean Daily Return  Daily Standard Deviation  Number of Days  \
Market_Regime                                                                
Unknown                 0.235358                   2.21087             515   

               Mean Regime Length  
Market_Regime                      
Unknown                       NaN  


C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_47776\1952905365.py:840: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Market_3yr_CAGR'] = rolling_3yr_cagr(data['Market_Value'])
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_47776\1952905365.py:841: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'Strategy_3yr_CAGR'] = rolling_3yr_cagr(data['Portfolio_Value'])



Worst 3-Year Periods for Each Decade (Market Downturns):
           Start_Date   End_Date  Market_3yr_CAGR  Strategy_3yr_CAGR  \
Date                                                                   
2015-01-01 2015-01-01 2015-01-01              NaN                NaN   
2020-01-02 2017-01-05 2020-01-02              NaN                NaN   

            Start_Strategy_Value  End_Strategy_Value  Start_Market_Value  \
Date                                                                       
2015-01-01         100000.000000       100000.000000       100000.000000   
2020-01-02         112311.567251       160573.079107       112311.567251   

            End_Market_Value  
Date                          
2015-01-01     100000.000000  
2020-01-02     160573.079107  

Performance Statistics:
        Statistic       Strategy         Market
             CAGR 11.6246247249% 11.6246247249%
     Max Drawdown        -29.18%        -29.18%
     Sharpe Ratio           0.58           0.58
    Sor